In [136]:
import pymysql, os
from sqlalchemy import create_engine, inspect, func, or_
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, aliased
import pandas as pd

In [2]:
pymysql.install_as_MySQLdb()

In [3]:
# Connect to MySQL database
PASSWD = os.getenv("LOCAL_MYSQL_PASS")
USER = "root"
PORT = "3306"
DOMAIN = "localhost"
DB = "sakila"
engine = create_engine(f"mysql://{USER}:{PASSWD}@{DOMAIN}:{PORT}/{DB}")
conn = engine.connect()

In [4]:
Base = automap_base()

In [5]:
Base.prepare(engine, reflect=True)

/usr/local/anaconda3/lib/python3.7/site-packages/sqlalchemy/dialects/mysql/reflection.py:193: SAWarning: Did not recognize type 'geometry' of column 'location'
  "Did not recognize type '%s' of column '%s'" % (type_, name)


In [6]:
Base.classes.keys()

['actor',
 'address',
 'city',
 'country',
 'category',
 'customer',
 'store',
 'staff',
 'film',
 'language',
 'film_actor',
 'film_category',
 'film_text',
 'inventory',
 'payment',
 'rental']

In [7]:
inspector = inspect(engine)

In [8]:
inspector.get_table_names()

['actor',
 'address',
 'category',
 'city',
 'country',
 'customer',
 'film',
 'film_actor',
 'film_category',
 'film_text',
 'inventory',
 'language',
 'payment',
 'rental',
 'staff',
 'store']

In [9]:
columns = inspector.get_columns('actor')
for column in columns:
    print(column["name"], column["type"])

actor_id SMALLINT(5) UNSIGNED
first_name VARCHAR(45)
last_name VARCHAR(45)
last_update TIMESTAMP
actor_name VARCHAR(50)


In [10]:
session = Session(engine)

-- 1a. Display the first and last names of all actors from the table actor.

In [11]:
Actor = Base.classes.actor

In [12]:
for instance in session.query(Actor).all():
    print(instance.first_name, instance.last_name)

PENELOPE GUINESS
NICK WAHLBERG
ED CHASE
JENNIFER DAVIS
JOHNNY LOLLOBRIGIDA
BETTE NICHOLSON
GRACE MOSTEL
MATTHEW JOHANSSON
JOE SWANK
CHRISTIAN GABLE
ZERO CAGE
KARL BERRY
UMA WOOD
VIVIEN BERGEN
CUBA OLIVIER
FRED COSTNER
HELEN VOIGHT
DAN TORN
BOB FAWCETT
LUCILLE TRACY
KIRSTEN PALTROW
ELVIS MARX
SANDRA KILMER
CAMERON STREEP
KEVIN BLOOM
RIP CRAWFORD
JULIA MCQUEEN
WOODY HOFFMAN
ALEC WAYNE
SANDRA PECK
SISSY SOBIESKI
TIM HACKMAN
MILLA PECK
AUDREY OLIVIER
JUDY DEAN
BURT DUKAKIS
VAL BOLGER
TOM MCKELLEN
GOLDIE BRODY
JOHNNY CAGE
JODIE DEGENERES
TOM MIRANDA
KIRK JOVOVICH
NICK STALLONE
REESE KILMER
PARKER GOLDBERG
JULIA BARRYMORE
FRANCES DAY-LEWIS
ANNE CRONYN
NATALIE HOPKINS
GARY PHOENIX
CARMEN HUNT
MENA TEMPLE
PENELOPE PINKETT
FAY KILMER
DAN HARRIS
JUDE CRUISE
CHRISTIAN AKROYD
DUSTIN TAUTOU
HENRY BERRY
CHRISTIAN NEESON
JAYNE NEESON
CAMERON WRAY
RAY JOHANSSON
ANGELA HUDSON
MARY TANDY
JESSICA BAILEY
RIP WINSLET
KENNETH PALTROW
MICHELLE MCCONAUGHEY
ADAM GRANT
SEAN WILLIAMS
GARY PENN
MILLA KEITEL
BURT 

-- 1b. Display the first and last name of each actor in a single column in upper case letters. Name the column Actor Name.

In [19]:
df = pd.DataFrame({
    "Actor Name": [item[0] + " " + item[1] for item in session.query(Actor.first_name, Actor.last_name).all()]
})

In [20]:
df.head()

,Actor Name
0,PENELOPE GUINESS
1,NICK WAHLBERG
2,ED CHASE
3,JENNIFER DAVIS
4,JOHNNY LOLLOBRIGIDA


-- 2a. You need to find the ID number, first name, and last name of an actor, of whom you know only the first name, "Joe." 
-- What is one query would you use to obtain this information?

In [24]:
for instance in session.query(Actor).filter(Actor.first_name == "joe"):
    print(instance.actor_id, instance.first_name, instance.last_name)

9 JOE SWANK


-- 2b. Find all actors whose last name contain the letters GEN:

In [25]:
for instance in session.query(Actor).filter(Actor.last_name.like('%GEN%')):
    print(instance.actor_id, instance.first_name, instance.last_name)

14 VIVIEN BERGEN
41 JODIE DEGENERES
107 GINA DEGENERES
166 NICK DEGENERES


-- 2c. Find all actors whose last names contain the letters LI. This time, order the rows by last name and first name, in that order:

In [26]:
for instance in session.query(Actor).filter(Actor.last_name.like('%LI%')).order_by(Actor.last_name, Actor.first_name):
    print(instance.actor_id, instance.first_name, instance.last_name)

86 GREG CHAPLIN
82 WOODY JOLIE
34 AUDREY OLIVIER
15 CUBA OLIVIER
172 GROUCHO WILLIAMS
137 MORGAN WILLIAMS
72 SEAN WILLIAMS
83 BEN WILLIS
96 GENE WILLIS
164 HUMPHREY WILLIS


-- 2d. Using IN, display the country_id and country columns of the following countries: Afghanistan, Bangladesh, and China:

In [27]:
Country = Base.classes.country

In [28]:
columns = inspector.get_columns('country')
for column in columns:
    print(column["name"], column["type"])

country_id SMALLINT(5) UNSIGNED
country VARCHAR(50)
last_update TIMESTAMP


In [29]:
for instance in session.query(Country).filter(Country.country.in_(['Afghanistan', 'Bangladesh', 'China'])):
    print(instance.country_id, instance.country)

1 Afghanistan
12 Bangladesh
23 China


-- 3a. You want to keep a description of each actor. 
-- You don't think you will be performing queries on a description, so create a column in the table actor named description and use the data type BLOB (Make sure to research the type BLOB, as the difference between it and VARCHAR are significant).

-- 3b. Very quickly you realize that entering descriptions for each actor is too much effort. Delete the description column.

-- 4a. List the last names of actors, as well as how many actors have that last name.

In [34]:
for item in session.query(Actor.last_name, func.count(Actor.last_name)).group_by(Actor.last_name).all():
    print(item[0], item[1])

AKROYD 3
ALLEN 3
ASTAIRE 1
BACALL 1
BAILEY 2
BALE 1
BALL 1
BARRYMORE 1
BASINGER 1
BENING 2
BERGEN 1
BERGMAN 1
BERRY 3
BIRCH 1
BLOOM 1
BOLGER 2
BRIDGES 1
BRODY 2
BULLOCK 1
CAGE 2
CARREY 1
CHAPLIN 1
CHASE 2
CLOSE 1
COSTNER 1
CRAWFORD 2
CRONYN 2
CROWE 1
CRUISE 1
CRUZ 1
DAMON 1
DAVIS 3
DAY-LEWIS 1
DEAN 2
DEE 2
DEGENERES 3
DENCH 2
DEPP 2
DERN 1
DREYFUSS 1
DUKAKIS 2
DUNST 1
FAWCETT 2
GABLE 1
GARLAND 3
GIBSON 1
GOLDBERG 1
GOODING 2
GRANT 1
GUINESS 3
HACKMAN 2
HARRIS 3
HAWKE 1
HESTON 1
HOFFMAN 3
HOPE 1
HOPKINS 3
HOPPER 2
HUDSON 1
HUNT 1
HURT 1
JACKMAN 2
JOHANSSON 3
JOLIE 1
JOVOVICH 1
KEITEL 3
KILMER 5
LEIGH 1
LOLLOBRIGIDA 1
MALDEN 1
MANSFIELD 1
MARX 1
MCCONAUGHEY 2
MCDORMAND 1
MCKELLEN 2
MCQUEEN 2
MIRANDA 1
MONROE 2
MOSTEL 2
NEESON 2
NICHOLSON 1
NOLTE 4
OLIVIER 2
PALTROW 2
PECK 3
PENN 2
PESCI 1
PFEIFFER 1
PHOENIX 1
PINKETT 1
PITT 1
POSEY 1
PRESLEY 1
REYNOLDS 1
RYDER 1
SILVERSTONE 2
SINATRA 1
SOBIESKI 1
STALLONE 1
STREEP 2
SUVARI 1
SWANK 1
TANDY 2
TAUTOU 1
TEMPLE 4
TOMEI 1
TORN 3
TRACY 2
VOIGHT

-- 4b. List last names of actors and the number of actors who have that last name, but only for names that are shared by at least two actors.

In [36]:
for item in session.query(Actor.last_name, func.count(Actor.last_name)).group_by(Actor.last_name) \
.having(func.count(Actor.last_name) > 1):
    print(item[0], item[1])

AKROYD 3
ALLEN 3
BAILEY 2
BENING 2
BERRY 3
BOLGER 2
BRODY 2
CAGE 2
CHASE 2
CRAWFORD 2
CRONYN 2
DAVIS 3
DEAN 2
DEE 2
DEGENERES 3
DENCH 2
DEPP 2
DUKAKIS 2
FAWCETT 2
GARLAND 3
GOODING 2
GUINESS 3
HACKMAN 2
HARRIS 3
HOFFMAN 3
HOPKINS 3
HOPPER 2
JACKMAN 2
JOHANSSON 3
KEITEL 3
KILMER 5
MCCONAUGHEY 2
MCKELLEN 2
MCQUEEN 2
MONROE 2
MOSTEL 2
NEESON 2
NOLTE 4
OLIVIER 2
PALTROW 2
PECK 3
PENN 2
SILVERSTONE 2
STREEP 2
TANDY 2
TEMPLE 4
TORN 3
TRACY 2
WAHLBERG 2
WEST 2
WILLIAMS 3
WILLIS 3
WINSLET 2
WOOD 2
ZELLWEGER 3


-- 4c. The actor HARPO WILLIAMS was accidentally entered in the actor table as GROUCHO WILLIAMS. Write a query to fix the record.

In [47]:
session.query(Actor).filter(Actor.first_name == 'GROUCHO', Actor.last_name == 'WILLIAMS').update({
    'first_name': 'HARPO'
})

1

In [48]:
for instance in session.query(Actor).filter(Actor.first_name == 'GROUCHO', Actor.last_name == 'WILLIAMS'):
    print(instance.first_name, instance.last_name)

In [59]:
actor = session.query(Actor).filter(Actor.first_name == 'GROUCHO', Actor.last_name == 'WILLIAMS').one_or_none()
print(actor)

None


In [49]:
for instance in session.query(Actor).filter(Actor.first_name == 'HARPO', Actor.last_name == 'WILLIAMS'):
    print(instance.first_name, instance.last_name)

HARPO WILLIAMS


In [50]:
session.query(Actor).filter(Actor.first_name == 'HARPO', Actor.last_name == 'WILLIAMS')

In [51]:
session.query(Actor).filter(Actor.first_name == 'HARPO', Actor.last_name == 'WILLIAMS').one()

In [52]:
session.query(Actor).filter(Actor.first_name == 'HARPO', Actor.last_name == 'WILLIAMS').one().last_name

'WILLIAMS'

In [53]:
session.query(Actor).filter(Actor.first_name == 'HARPO', Actor.last_name == 'WILLIAMS').all()[0].last_name

'WILLIAMS'

-- 4d. Perhaps we were too hasty in changing GROUCHO to HARPO. 
-- It turns out that GROUCHO was the correct name after all! In a single query, if the first name of the actor is currently HARPO, change it to GROUCHO.

In [60]:
actor = session.query(Actor).filter(Actor.first_name == 'HARPO').one_or_none()
print(actor)

In [61]:
actor.first_name = 'GROUCHO'
session.commit()

In [62]:
actor = session.query(Actor).filter(Actor.first_name == 'HARPO').one_or_none()
print(actor)

None


In [64]:
actor = session.query(Actor).filter(Actor.first_name == 'GROUCHO', Actor.last_name == 'WILLIAMS').one_or_none()
if actor:
    print('Yes!!!')

Yes!!!


In [65]:
actor_list = session.query(Actor).filter(Actor.first_name == 'HARPO').all()
print(actor_list)

[]


-- 6a. Use JOIN to display the first and last names, as well as the address, of each staff member. Use the tables staff and address:

In [81]:
Staff = Base.classes.staff
Address = Base.classes.address
City = Base.classes.city

In [67]:
columns = inspector.get_columns('staff')
for column in columns:
    print(column["name"], column["type"])

staff_id TINYINT(3) UNSIGNED
first_name VARCHAR(45)
last_name VARCHAR(45)
address_id SMALLINT(5) UNSIGNED
picture BLOB
email VARCHAR(50)
store_id TINYINT(3) UNSIGNED
active TINYINT(1)
username VARCHAR(16)
password VARCHAR(40) CHARACTER SET utf8 COLLATE utf8_bin
last_update TIMESTAMP


In [68]:
columns = inspector.get_columns('address')
for column in columns:
    print(column["name"], column["type"])

address_id SMALLINT(5) UNSIGNED
address VARCHAR(50)
address2 VARCHAR(50)
district VARCHAR(20)
city_id SMALLINT(5) UNSIGNED
postal_code VARCHAR(10)
phone VARCHAR(20)
location 

/usr/local/anaconda3/lib/python3.7/site-packages/sqlalchemy/dialects/mysql/reflection.py:193: SAWarning: Did not recognize type 'geometry' of column 'location'
  "Did not recognize type '%s' of column '%s'" % (type_, name)


CompileError: Can't generate DDL for NullType(); did you forget to specify a type on this Column?

In [82]:
columns = inspector.get_columns('city')
for column in columns:
    print(column["name"], column["type"])

city_id SMALLINT(5) UNSIGNED
city VARCHAR(50)
country_id SMALLINT(5) UNSIGNED
last_update TIMESTAMP


In [92]:
result = session.query(
    Staff.staff_id, 
    Staff.first_name, 
    Staff.last_name, 
    Address.address, 
    Address.district,
    City.city,
    Address.postal_code
) \
.outerjoin(Address, Staff.address_id == Address.address_id) \
.outerjoin(City, Address.city_id == City.city_id)

In [93]:
result

In [94]:
for row in result:
    print(*row)

1 Mike Hillyer 23 Workhaven Lane Alberta Lethbridge 
2 Jon Stephens 1411 Lillydale Drive QLD Woodridge 


In [95]:
session.query(Address.address, Address.postal_code).filter(Address.address == '23 Workhaven Lane').all()

[('23 Workhaven Lane', '')]

-- 6b. Use JOIN to display the total amount rung up by each staff member in August of 2005. Use tables staff and payment.

In [96]:
Payment = Base.classes.payment

In [97]:
columns = inspector.get_columns('payment')
for column in columns:
    print(column["name"], column["type"])

payment_id SMALLINT(5) UNSIGNED
customer_id SMALLINT(5) UNSIGNED
staff_id TINYINT(3) UNSIGNED
rental_id INTEGER(11)
amount DECIMAL(5, 2)
payment_date DATETIME
last_update TIMESTAMP


In [100]:
result = session.query(
    Staff.staff_id, 
    Staff.first_name, 
    Staff.last_name, 
    func.sum(Payment.amount)
) \
.join(Staff, Payment.staff_id == Staff.staff_id) \
.filter(func.month(Payment.payment_date) == 8) \
.group_by(Staff.staff_id)

In [101]:
for row in result:
    print(*row)

1 Mike Hillyer 11853.65
2 Jon Stephens 12218.48


-- 6c. List each film and the number of actors who are listed for that film. Use tables film_actor and film. Use inner join.

In [102]:
Film_Actor = Base.classes.film_actor
Film = Base.classes.film

In [103]:
columns = inspector.get_columns('film')
for column in columns:
    print(column["name"], column["type"])

film_id SMALLINT(5) UNSIGNED
title VARCHAR(255)
description TEXT
release_year YEAR(4)
language_id TINYINT(3) UNSIGNED
original_language_id TINYINT(3) UNSIGNED
rental_duration TINYINT(3) UNSIGNED
rental_rate DECIMAL(4, 2)
length SMALLINT(5) UNSIGNED
replacement_cost DECIMAL(5, 2)
rating ENUM('G','PG','PG-13','R','NC-17')
special_features SET('Trailers','Commentaries','Deleted Scenes','Behind the Scenes')
last_update TIMESTAMP


In [104]:
columns = inspector.get_columns('film_actor')
for column in columns:
    print(column["name"], column["type"])

actor_id SMALLINT(5) UNSIGNED
film_id SMALLINT(5) UNSIGNED
last_update TIMESTAMP


In [111]:
for row in session.query(Film_Actor).limit(50).all():
    print(row.actor_id, row.film_id)

1 1
1 23
1 25
1 106
1 140
1 166
1 277
1 361
1 438
1 499
1 506
1 509
1 605
1 635
1 749
1 832
1 939
1 970
1 980
2 3
2 31
2 47
2 105
2 132
2 145
2 226
2 249
2 314
2 321
2 357
2 369
2 399
2 458
2 481
2 485
2 518
2 540
2 550
2 555
2 561
2 742
2 754
2 811
2 958
3 17
3 40
3 42
3 87
3 111
3 185


In [112]:
result = session.query(
    Film.film_id,
    Film.title,
    func.count(Film_Actor.actor_id)
) \
.join(Film, Film_Actor.film_id == Film.film_id) \
.group_by(Film.film_id)

In [113]:
for row in result:
    print(*row)

1 ACADEMY DINOSAUR 10
2 ACE GOLDFINGER 4
3 ADAPTATION HOLES 5
4 AFFAIR PREJUDICE 5
5 AFRICAN EGG 5
6 AGENT TRUMAN 7
7 AIRPLANE SIERRA 5
8 AIRPORT POLLOCK 4
9 ALABAMA DEVIL 9
10 ALADDIN CALENDAR 8
11 ALAMO VIDEOTAPE 4
12 ALASKA PHANTOM 7
13 ALI FOREVER 5
14 ALICE FANTASIA 4
15 ALIEN CENTER 6
16 ALLEY EVOLUTION 5
17 ALONE TRIP 8
18 ALTER VICTORY 4
19 AMADEUS HOLY 6
20 AMELIE HELLFIGHTERS 6
21 AMERICAN CIRCUS 5
22 AMISTAD MIDSUMMER 4
23 ANACONDA CONFESSIONS 5
24 ANALYZE HOOSIERS 5
25 ANGELS LIFE 9
26 ANNIE IDENTITY 3
27 ANONYMOUS HUMAN 9
28 ANTHEM LUKE 2
29 ANTITRUST TOMATOES 7
30 ANYTHING SAVANNAH 3
31 APACHE DIVINE 4
32 APOCALYPSE FLAMINGOS 5
33 APOLLO TEEN 8
34 ARABIA DOGMA 12
35 ARACHNOPHOBIA ROLLERCOASTER 8
36 ARGONAUTS TOWN 5
37 ARIZONA BANG 4
38 ARK RIDGEMONT 3
39 ARMAGEDDON LOST 7
40 ARMY FLINTSTONES 7
41 ARSENIC INDEPENDENCE 3
42 ARTIST COLDBLOODED 7
43 ATLANTIS CAUSE 9
44 ATTACKS HATE 4
45 ATTRACTION NEWTON 4
46 AUTUMN CROW 3
47 BABY HALL 8
48 BACKLASH UNDEFEATED 7
49 BADMAN DAW

582 MIRACLE VIRTUAL 1
583 MISSION ZOOLANDER 5
584 MIXED DOORS 6
585 MOB DUFFEL 6
586 MOCKINGBIRD HOLLYWOOD 3
587 MOD SECRETARY 6
588 MODEL FISH 8
589 MODERN DORADO 5
590 MONEY HAROLD 7
591 MONSOON CAUSE 5
592 MONSTER SPARTACUS 7
593 MONTEREY LABYRINTH 6
594 MONTEZUMA COMMAND 2
595 MOON BUNCH 1
596 MOONSHINE CABIN 5
597 MOONWALKER FOOL 7
598 MOSQUITO ARMAGEDDON 4
599 MOTHER OLEANDER 5
600 MOTIONS DETAILS 8
601 MOULIN WAKE 3
602 MOURNING PURPLE 7
603 MOVIE SHAKESPEARE 8
604 MULAN MOON 5
605 MULHOLLAND BEAST 6
606 MUMMY CREATURES 13
607 MUPPET MILE 8
608 MURDER ANTITRUST 8
609 MUSCLE BRIGHT 5
610 MUSIC BOONDOCK 3
611 MUSKETEERS WAIT 1
612 MUSSOLINI SPOILERS 7
613 MYSTIC TRUMAN 4
614 NAME DETECTIVE 3
615 NASH CHOCOLAT 7
616 NATIONAL STORY 7
617 NATURAL STOCK 4
618 NECKLACE OUTBREAK 8
619 NEIGHBORS CHARADE 7
620 NEMO CAMPUS 5
621 NETWORK PEAK 3
622 NEWSIES STORY 8
623 NEWTON LABYRINTH 4
624 NIGHTMARE CHILL 2
625 NONE SPIKING 6
626 NOON PAPI 6
627 NORTH TEQUILA 5
628 NORTHWEST POLISH 8
629 N

-- 6d. How many copies of the film Hunchback Impossible exist in the inventory system?

In [114]:
Inventory = Base.classes.inventory

In [115]:
columns = inspector.get_columns('inventory')
for column in columns:
    print(column["name"], column["type"])

inventory_id MEDIUMINT(8) UNSIGNED
film_id SMALLINT(5) UNSIGNED
store_id TINYINT(3) UNSIGNED
last_update TIMESTAMP


In [116]:
len(
    session.query(Inventory.inventory_id) \
    .filter(Inventory.film_id == Film.film_id) \
    .filter(Film.title == 'Hunchback Impossible') \
    .all()
)

6

-- 6e. Using the tables payment and customer and the JOIN command, list the total paid by each customer. 
-- List the customers alphabetically by last name:

In [117]:
Customer = Base.classes.customer

In [118]:
columns = inspector.get_columns('customer')
for column in columns:
    print(column["name"], column["type"])

customer_id SMALLINT(5) UNSIGNED
store_id TINYINT(3) UNSIGNED
first_name VARCHAR(45)
last_name VARCHAR(45)
email VARCHAR(50)
address_id SMALLINT(5) UNSIGNED
active TINYINT(1)
create_date DATETIME
last_update TIMESTAMP


In [126]:
result = session.query(
    Customer.customer_id,
    Customer.first_name,
    Customer.last_name,
    func.sum(Payment.amount).label('total_amount')
) \
.join(Customer, Payment.customer_id == Customer.customer_id) \
.group_by(Payment.customer_id) \
.order_by(Customer.last_name, Customer.first_name)

In [127]:
result.column_descriptions

[{'name': 'customer_id',
  'type': SMALLINT(display_width=5, unsigned=True),
  'aliased': False,
  'expr': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x10a324360>,
  'entity': sqlalchemy.ext.automap.customer},
 {'name': 'first_name',
  'type': VARCHAR(length=45),
  'aliased': False,
  'expr': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x10a3244c0>,
  'entity': sqlalchemy.ext.automap.customer},
 {'name': 'last_name',
  'type': VARCHAR(length=45),
  'aliased': False,
  'expr': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x10a324570>,
  'entity': sqlalchemy.ext.automap.customer},
 {'name': 'total_amount',
  'type': DECIMAL(precision=5, scale=2),
  'aliased': False,
  'expr': <sqlalchemy.sql.elements.Label object at 0x11ec39a20>,
  'entity': sqlalchemy.ext.automap.payment}]

In [122]:
for row in result:
    print(*row)

505 RAFAEL ABNEY 97.79
504 NATHANIEL ADAM 133.72
36 KATHLEEN ADAMS 92.73
96 DIANA ALEXANDER 105.73
470 GORDON ALLARD 160.68
27 SHIRLEY ALLEN 126.69
220 CHARLENE ALVAREZ 114.73
11 LISA ANDERSON 106.76
326 JOSE ANDREW 96.75
183 IDA ANDREWS 76.77
449 OSCAR AQUINO 99.80
368 HARRY ARCE 157.65
560 JORDAN ARCHULETA 132.70
188 MELANIE ARMSTRONG 92.75
170 BEATRICE ARNOLD 119.74
591 KENT ARSENAULT 134.73
345 CARL ARTIS 106.77
530 DARRYL ASHCRAFT 76.77
540 TYRONE ASHER 112.76
196 ALMA AUSTIN 151.65
60 MILDRED BAILEY 98.75
37 PAMELA BAKER 95.77
383 MARTIN BALES 103.73
559 EVERETT BANDA 110.72
215 JESSIE BANKS 91.74
551 CLAYTON BARBEE 96.74
503 ANGEL BARCLAY 115.68
362 NICHOLAS BARFIELD 145.68
382 VICTOR BARKLEY 91.76
79 RACHEL BARNES 84.78
275 CAROLE BARNETT 108.70
280 TRACEY BARRETT 118.73
295 DAISY BATES 162.62
317 EDWARD BAUGH 114.72
301 ROBERT BAUGHMAN 92.79
58 JEAN BELL 115.73
511 CHESTER BENNER 99.76
77 JANE BENNETT 100.72
168 REGINA BERRY 135.66
495 CHARLIE BESS 120.74
448 MIGUEL BETANCOURT

129 CARRIE PORTER 124.66
371 BILLY POULIN 149.65
85 ANNE POWELL 87.77
463 DARRELL POWER 91.75
563 KEN PREWITT 110.71
76 IRENE PRICE 77.77
415 GLENN PULLEN 93.77
333 ANDREW PURDY 109.73
332 STEPHEN QUALLS 118.72
436 TROY QUIGLEY 144.70
348 ROGER QUINTANILLA 146.64
414 VINCENT RALSTON 105.75
70 CHRISTINA RAMIREZ 80.82
140 EVA RAMOS 83.82
515 ANDRE RAPP 126.72
407 DALE RATCLIFF 112.73
173 AUDREY RAY 119.71
451 JIM REA 128.67
55 DORIS REED 100.78
232 CONSTANCE REID 95.75
532 NEIL RENNER 152.68
141 DEBBIE REYES 130.68
112 ROSA REYNOLDS 133.70
507 EDGAR RHOADS 95.75
297 SHERRI RHODES 128.67
146 JAMIE RICE 139.71
212 WILMA RICHARDS 91.80
63 ASHLEY RICHARDSON 112.75
456 RONNIE RICKETTS 100.75
357 KEITH RICO 89.74
187 BRITTANY RILEY 159.72
312 MARK RINEHART 104.74
61 KATHERINE RIVERA 58.86
450 JAY ROBB 89.74
331 ERIC ROBERT 122.73
43 CHRISTINE ROBERTS 99.76
147 JOANNE ROBERTSON 127.66
472 GREG ROBINS 141.70
20 SHARON ROBINSON 115.70
22 LAURA RODRIGUEZ 113.78
289 VIOLET RODRIQUEZ 142.70
54 TERES

-- 7a. The music of Queen and Kris Kristofferson have seen an unlikely resurgence. 
-- As an unintended consequence, films starting with the letters K and Q have also soared in popularity. 
-- Use subqueries to display the titles of movies starting with the letters K and Q whose language is English.

In [123]:
Language = Base.classes.language

In [124]:
columns = inspector.get_columns('language')
for column in columns:
    print(column["name"], column["type"])

language_id TINYINT(3) UNSIGNED
name CHAR(20)
last_update TIMESTAMP


In [129]:
result = session.query(
    Film.film_id,
    Film.title,
    Language.name
) \
.join(Language, Film.language_id == Language.language_id) \
.filter(or_(Film.title.like('K%'), Film.title.like('Q%'))) \
.filter(Language.name == 'English')

In [130]:
for row in result:
    print(*row)

493 KANE EXORCIST English
494 KARATE MOON English
495 KENTUCKIAN GIANT English
496 KICK SAVANNAH English
497 KILL BROTHERHOOD English
498 KILLER INNOCENT English
499 KING EVOLUTION English
500 KISS GLORY English
501 KISSING DOLLS English
502 KNOCK WARLOCK English
503 KRAMER CHOCOLATE English
504 KWAI HOMEWARD English
706 QUEEN LUKE English
707 QUEST MUSSOLINI English
708 QUILLS BULL English


-- 7b. Use subqueries to display all actors who appear in the film Alone Trip.

In [131]:
columns = inspector.get_columns('actor')
for column in columns:
    print(column["name"], column["type"])

actor_id SMALLINT(5) UNSIGNED
first_name VARCHAR(45)
last_name VARCHAR(45)
last_update TIMESTAMP
actor_name VARCHAR(50)


In [132]:
columns = inspector.get_columns('film_actor')
for column in columns:
    print(column["name"], column["type"])

actor_id SMALLINT(5) UNSIGNED
film_id SMALLINT(5) UNSIGNED
last_update TIMESTAMP


In [133]:
columns = inspector.get_columns('film')
for column in columns:
    print(column["name"], column["type"])

film_id SMALLINT(5) UNSIGNED
title VARCHAR(255)
description TEXT
release_year YEAR(4)
language_id TINYINT(3) UNSIGNED
original_language_id TINYINT(3) UNSIGNED
rental_duration TINYINT(3) UNSIGNED
rental_rate DECIMAL(4, 2)
length SMALLINT(5) UNSIGNED
replacement_cost DECIMAL(5, 2)
rating ENUM('G','PG','PG-13','R','NC-17')
special_features SET('Trailers','Commentaries','Deleted Scenes','Behind the Scenes')
last_update TIMESTAMP


In [139]:
result = session.query(
    Actor.actor_id,
    Actor.first_name,
    Actor.last_name,
    Film.title
) \
.join(Film_Actor, Actor.actor_id == Film_Actor.actor_id) \
.join(Film, Film_Actor.film_id == Film.film_id) \
.filter(Film.title == 'Alone Trip')

In [140]:
for row in result:
    print(*row)

3 ED CHASE ALONE TRIP
12 KARL BERRY ALONE TRIP
13 UMA WOOD ALONE TRIP
82 WOODY JOLIE ALONE TRIP
100 SPENCER DEPP ALONE TRIP
160 CHRIS DEPP ALONE TRIP
167 LAURENCE BULLOCK ALONE TRIP
187 RENEE BALL ALONE TRIP


In [141]:
# Using filter only
result = session.query(
    Actor.actor_id,
    Actor.first_name,
    Actor.last_name,
    Film.title
) \
.filter(Actor.actor_id == Film_Actor.actor_id) \
.filter(Film_Actor.film_id == Film.film_id) \
.filter(Film.title == 'Alone Trip')

In [142]:
for row in result:
    print(*row)

3 ED CHASE ALONE TRIP
12 KARL BERRY ALONE TRIP
13 UMA WOOD ALONE TRIP
82 WOODY JOLIE ALONE TRIP
100 SPENCER DEPP ALONE TRIP
160 CHRIS DEPP ALONE TRIP
167 LAURENCE BULLOCK ALONE TRIP
187 RENEE BALL ALONE TRIP


In [143]:
# Using subquery
film_select_stmt = session.query(Film.film_id).filter(Film.title == 'Alone Trip').subquery()
film_actor_select_stmt = session.query(Film_Actor.actor_id).filter(Film_Actor.film_id == film_select_stmt.c.film_id).subquery()
result = session.query(Actor.actor_name).filter(Actor.actor_id == film_actor_select_stmt.c.actor_id)

In [144]:
for row in result:
    print(*row)

ED CHASE
KARL BERRY
UMA WOOD
WOODY JOLIE
SPENCER DEPP
CHRIS DEPP
LAURENCE BULLOCK
RENEE BALL


-- 7c. You want to run an email marketing campaign in Canada, for which you will need the names and email addresses of all Canadian customers. 
-- Use joins to retrieve this information.

In [145]:
columns = inspector.get_columns('customer')
for column in columns:
    print(column["name"], column["type"])

customer_id SMALLINT(5) UNSIGNED
store_id TINYINT(3) UNSIGNED
first_name VARCHAR(45)
last_name VARCHAR(45)
email VARCHAR(50)
address_id SMALLINT(5) UNSIGNED
active TINYINT(1)
create_date DATETIME
last_update TIMESTAMP


In [147]:
columns = inspector.get_columns('address')
for column in columns:
    print(column["name"])

address_id
address
address2
district
city_id
postal_code
phone
location
last_update


In [148]:
columns = inspector.get_columns('city')
for column in columns:
    print(column["name"], column["type"])

city_id SMALLINT(5) UNSIGNED
city VARCHAR(50)
country_id SMALLINT(5) UNSIGNED
last_update TIMESTAMP


In [149]:
columns = inspector.get_columns('country')
for column in columns:
    print(column["name"], column["type"])

country_id SMALLINT(5) UNSIGNED
country VARCHAR(50)
last_update TIMESTAMP


In [150]:
result = session.query(
    Customer.customer_id,
    Customer.first_name,
    Customer.last_name,
    Customer.email
) \
.join(Address, Customer.address_id == Address.address_id) \
.join(City, Address.city_id == City.city_id) \
.join(Country, City.country_id == Country.country_id)

In [151]:
for row in result:
    print(*row)

1 MARY SMITH MARY.SMITH@sakilacustomer.org
2 PATRICIA JOHNSON PATRICIA.JOHNSON@sakilacustomer.org
3 LINDA WILLIAMS LINDA.WILLIAMS@sakilacustomer.org
4 BARBARA JONES BARBARA.JONES@sakilacustomer.org
5 ELIZABETH BROWN ELIZABETH.BROWN@sakilacustomer.org
6 JENNIFER DAVIS JENNIFER.DAVIS@sakilacustomer.org
7 MARIA MILLER MARIA.MILLER@sakilacustomer.org
8 SUSAN WILSON SUSAN.WILSON@sakilacustomer.org
9 MARGARET MOORE MARGARET.MOORE@sakilacustomer.org
10 DOROTHY TAYLOR DOROTHY.TAYLOR@sakilacustomer.org
11 LISA ANDERSON LISA.ANDERSON@sakilacustomer.org
12 NANCY THOMAS NANCY.THOMAS@sakilacustomer.org
13 KAREN JACKSON KAREN.JACKSON@sakilacustomer.org
14 BETTY WHITE BETTY.WHITE@sakilacustomer.org
15 HELEN HARRIS HELEN.HARRIS@sakilacustomer.org
16 SANDRA MARTIN SANDRA.MARTIN@sakilacustomer.org
17 DONNA THOMPSON DONNA.THOMPSON@sakilacustomer.org
18 CAROL GARCIA CAROL.GARCIA@sakilacustomer.org
19 RUTH MARTINEZ RUTH.MARTINEZ@sakilacustomer.org
20 SHARON ROBINSON SHARON.ROBINSON@sakilacustomer.org
21 MI

420 JACOB LANCE JACOB.LANCE@sakilacustomer.org
421 LEE HAWKS LEE.HAWKS@sakilacustomer.org
422 MELVIN ELLINGTON MELVIN.ELLINGTON@sakilacustomer.org
423 ALFRED CASILLAS ALFRED.CASILLAS@sakilacustomer.org
424 KYLE SPURLOCK KYLE.SPURLOCK@sakilacustomer.org
425 FRANCIS SIKES FRANCIS.SIKES@sakilacustomer.org
426 BRADLEY MOTLEY BRADLEY.MOTLEY@sakilacustomer.org
427 JESUS MCCARTNEY JESUS.MCCARTNEY@sakilacustomer.org
428 HERBERT KRUGER HERBERT.KRUGER@sakilacustomer.org
429 FREDERICK ISBELL FREDERICK.ISBELL@sakilacustomer.org
430 RAY HOULE RAY.HOULE@sakilacustomer.org
431 JOEL FRANCISCO JOEL.FRANCISCO@sakilacustomer.org
432 EDWIN BURK EDWIN.BURK@sakilacustomer.org
433 DON BONE DON.BONE@sakilacustomer.org
434 EDDIE TOMLIN EDDIE.TOMLIN@sakilacustomer.org
435 RICKY SHELBY RICKY.SHELBY@sakilacustomer.org
436 TROY QUIGLEY TROY.QUIGLEY@sakilacustomer.org
437 RANDALL NEUMANN RANDALL.NEUMANN@sakilacustomer.org
438 BARRY LOVELACE BARRY.LOVELACE@sakilacustomer.org
439 ALEXANDER FENNELL ALEXANDER.FENNELL@s

-- 7d. Sales have been lagging among young families, and you wish to target all family movies for a promotion. 
-- Identify all movies categorized as family films.

In [153]:
Film_Category = Base.classes.film_category
Category = Base.classes.category

In [154]:
columns = inspector.get_columns('film')
for column in columns:
    print(column["name"], column["type"])

film_id SMALLINT(5) UNSIGNED
title VARCHAR(255)
description TEXT
release_year YEAR(4)
language_id TINYINT(3) UNSIGNED
original_language_id TINYINT(3) UNSIGNED
rental_duration TINYINT(3) UNSIGNED
rental_rate DECIMAL(4, 2)
length SMALLINT(5) UNSIGNED
replacement_cost DECIMAL(5, 2)
rating ENUM('G','PG','PG-13','R','NC-17')
special_features SET('Trailers','Commentaries','Deleted Scenes','Behind the Scenes')
last_update TIMESTAMP


In [156]:
columns = inspector.get_columns('film_category')
for column in columns:
    print(column["name"], column["type"])

film_id SMALLINT(5) UNSIGNED
category_id TINYINT(3) UNSIGNED
last_update TIMESTAMP


In [157]:
columns = inspector.get_columns('category')
for column in columns:
    print(column["name"], column["type"])

category_id TINYINT(3) UNSIGNED
name VARCHAR(25)
last_update TIMESTAMP


In [158]:
result = session.query(
    Film.film_id,
    Film.title
) \
.join(Film_Category, Film.film_id == Film_Category.film_id) \
.join(Category, Film_Category.category_id == Category.category_id) \
.filter(Category.name == 'Family')

In [159]:
for row in result:
    print(*row)

5 AFRICAN EGG
31 APACHE DIVINE
43 ATLANTIS CAUSE
50 BAKED CLEOPATRA
53 BANG KWAI
63 BEDAZZLED MARRIED
71 BILKO ANONYMOUS
80 BLANKET BEVERLY
82 BLOOD ARGONAUTS
83 BLUES INSTINCT
94 BRAVEHEART HUMAN
139 CHASING FIGHT
145 CHISUM BEHAVIOR
147 CHOCOLAT HARRY
175 CONFUSED CANDLES
183 CONVERSATION DOWNHILL
213 DATE SPEED
231 DINOSAUR SECRETARY
262 DUMBO LUST
269 EARRING INSTINCT
273 EFFECT GLADIATOR
309 FEUD FROGMEN
315 FINDING ANACONDA
345 GABLES METROPOLIS
348 GANDHI KWAI
359 GLADIATOR WESTWARD
377 GREASE YOUTH
391 HALF OUTFIELD
419 HOCUS FRIDA
428 HOMICIDE PEACH
437 HOUSE DYNAMITE
442 HUNTING MUSKETEERS
458 INDIAN LOVE
476 JASON TRAP
479 JEDI BENEATH
498 KILLER INNOCENT
499 KING EVOLUTION
528 LOLITA WORLD
534 LOUISIANA HARRY
550 MAGUIRE APACHE
557 MANCHURIAN CURTAIN
603 MOVIE SHAKESPEARE
610 MUSIC BOONDOCK
617 NATURAL STOCK
621 NETWORK PEAK
634 ODDS BOOGIE
639 OPPOSITE NECKLACE
679 PILOT HOOSIERS
682 PITTSBURGH HUNCHBACK
695 PRESIDENT BANG
700 PRIX UNDEFEATED
710 RAGE GAMES
715 RANGE MOONW

-- 7e. Display the most frequently rented movies in descending order.

In [160]:
Rental = Base.classes.rental

In [161]:
columns = inspector.get_columns('rental')
for column in columns:
    print(column["name"], column["type"])

rental_id INTEGER(11)
rental_date DATETIME
inventory_id MEDIUMINT(8) UNSIGNED
customer_id SMALLINT(5) UNSIGNED
return_date DATETIME
staff_id TINYINT(3) UNSIGNED
last_update TIMESTAMP


In [162]:
columns = inspector.get_columns('inventory')
for column in columns:
    print(column["name"], column["type"])

inventory_id MEDIUMINT(8) UNSIGNED
film_id SMALLINT(5) UNSIGNED
store_id TINYINT(3) UNSIGNED
last_update TIMESTAMP


In [163]:
columns = inspector.get_columns('film')
for column in columns:
    print(column["name"], column["type"])

film_id SMALLINT(5) UNSIGNED
title VARCHAR(255)
description TEXT
release_year YEAR(4)
language_id TINYINT(3) UNSIGNED
original_language_id TINYINT(3) UNSIGNED
rental_duration TINYINT(3) UNSIGNED
rental_rate DECIMAL(4, 2)
length SMALLINT(5) UNSIGNED
replacement_cost DECIMAL(5, 2)
rating ENUM('G','PG','PG-13','R','NC-17')
special_features SET('Trailers','Commentaries','Deleted Scenes','Behind the Scenes')
last_update TIMESTAMP


In [190]:
result = session.query(
    Film.film_id,
    Film.title,
    func.count(Rental.rental_id).label('total_num_rentals')
) \
.join(Inventory, Rental.inventory_id == Inventory.inventory_id) \
.join(Film, Inventory.film_id == Film.film_id) \
.group_by(Film.film_id) \
.order_by(func.count(Film.film_id).desc())

In [191]:
for row in result:
    print(*row)

103 BUCKET BROTHERHOOD 34
738 ROCKETEER MOTHER 33
767 SCALAWAG DUCK 32
331 FORWARD TEMPLE 32
382 GRIT CLOCKWORK 32
489 JUGGLER HARDLY 32
730 RIDGEMONT SUBMARINE 32
31 APACHE DIVINE 31
891 TIMBERLAND SKY 31
1000 ZORRO ARK 31
735 ROBBERS JOON 31
753 RUSH GOODFELLAS 31
621 NETWORK PEAK 31
973 WIFE TURN 31
418 HOBBIT ALIEN 31
369 GOODFELLAS SALUTE 31
109 BUTTERFLY CHOCOLAT 30
403 HARRY IDAHO 30
559 MARRIED GO 30
239 DOGMA FAMILY 30
609 MUSCLE BRIGHT 30
127 CAT CONEHEADS 30
702 PULP BEVERLY 30
869 SUSPECTS QUILLS 30
979 WITCHES PANIC 30
341 FROST HEAD 30
374 GRAFFITI LOVE 30
563 MASSACRE USUAL 30
450 IDOLS SNATCHERS 30
748 RUGRATS SHAKESPEARE 30
789 SHOCK CABIN 30
285 ENGLISH BULWORTH 30
378 GREATEST NORTH 29
849 STORM HAPPINESS 29
875 TALENTED HOMICIDE 29
941 VIDEOTAPE ARSENIC 29
893 TITANS JERK 29
86 BOOGIE AMELIE 29
595 MOON BUNCH 29
945 VIRGINIAN PLUTO 29
174 CONFIDENTIAL INTERVIEW 29
73 BINGO TALENTED 29
284 ENEMY ODDS 29
220 DEER VIRGINIAN 29
301 FAMILY SWEET 29
361 GLEAMING JAWBREAKE

917 TUXEDO MILE 13
34 ARABIA DOGMA 13
145 CHISUM BEHAVIOR 13
296 EXPRESS LONELY 13
321 FLASH WARS 13
423 HOLLYWOOD ANONYMOUS 13
487 JINGLE SAGEBRUSH 13
577 MILE MULAN 13
593 MONTEREY LABYRINTH 13
736 ROBBERY BRIGHT 13
994 WYOMING STORM 13
440 HUNGER ROOF 13
660 PARTY KNOCK 13
711 RAGING AIRPLANE 13
762 SASSY PACKER 13
828 SPIKING ELEMENT 13
27 ANONYMOUS HUMAN 13
63 BEDAZZLED MARRIED 13
71 BILKO ANONYMOUS 13
96 BREAKING HOME 13
258 DRUMS DYNAMITE 13
290 EVERYONE CRAFT 13
449 IDENTITY LOVER 13
661 PAST SUICIDES 13
829 SPINAL ROCKY 13
209 DARKNESS WAR 13
540 LUCKY FLYING 13
604 MULAN MOON 13
705 PURPLE MOVIE 13
929 USUAL UNTOUCHABLES 13
65 BEHAVIOR RUNAWAY 13
98 BRIGHT ENCOUNTERS 13
202 DADDY PITTSBURGH 13
573 MICROCOSMOS PARADISE 13
597 MOONWALKER FOOL 13
765 SATURN NAME 13
133 CHAMBER ITALIAN 13
151 CINCINATTI WHISPERER 13
293 EXORCIST STING 13
639 OPPOSITE NECKLACE 13
882 TENENBAUMS COMMAND 13
194 CROW GREASE 12
519 LIBERTY MAGNIFICENT 12
567 MEET CHOCOLATE 12
825 SPEAKEASY DATE 12
93 

-- 7f. Write a query to display how much business, in dollars, each store brought in.

In [192]:
columns = inspector.get_columns('payment')
for column in columns:
    print(column["name"], column["type"])

payment_id SMALLINT(5) UNSIGNED
customer_id SMALLINT(5) UNSIGNED
staff_id TINYINT(3) UNSIGNED
rental_id INTEGER(11)
amount DECIMAL(5, 2)
payment_date DATETIME
last_update TIMESTAMP


In [193]:
columns = inspector.get_columns('customer')
for column in columns:
    print(column["name"], column["type"])

customer_id SMALLINT(5) UNSIGNED
store_id TINYINT(3) UNSIGNED
first_name VARCHAR(45)
last_name VARCHAR(45)
email VARCHAR(50)
address_id SMALLINT(5) UNSIGNED
active TINYINT(1)
create_date DATETIME
last_update TIMESTAMP


In [194]:
result = session.query(
    Customer.store_id,
    func.sum(Payment.amount).label('total_payment')
) \
.join(Customer, Payment.customer_id == Customer.customer_id) \
.group_by(Customer.store_id)

In [195]:
for row in result:
    print(*row)

1 37001.52
2 30414.99


-- 7g. Write a query to display for each store its store ID, city, and country.

In [196]:
Store = Base.classes.store

In [197]:
columns = inspector.get_columns('store')
for column in columns:
    print(column["name"], column["type"])

store_id TINYINT(3) UNSIGNED
manager_staff_id TINYINT(3) UNSIGNED
address_id SMALLINT(5) UNSIGNED
last_update TIMESTAMP


In [199]:
columns = inspector.get_columns('address')
for column in columns:
    print(column["name"])

address_id
address
address2
district
city_id
postal_code
phone
location
last_update


In [200]:
columns = inspector.get_columns('city')
for column in columns:
    print(column["name"], column["type"])

city_id SMALLINT(5) UNSIGNED
city VARCHAR(50)
country_id SMALLINT(5) UNSIGNED
last_update TIMESTAMP


In [201]:
columns = inspector.get_columns('country')
for column in columns:
    print(column["name"], column["type"])

country_id SMALLINT(5) UNSIGNED
country VARCHAR(50)
last_update TIMESTAMP


In [202]:
result = session.query(
    Store.store_id,
    City.city,
    Country.country
) \
.join(Address, Store.address_id == Address.address_id) \
.join(City, Address.city_id == City.city_id) \
.join(Country, City.country_id == Country.country_id)

In [203]:
for row in result:
    print(*row)

1 Lethbridge Canada
2 Woodridge Australia


-- 7h. List the top five genres in gross revenue in descending order. 
-- (Hint: you may need to use the following tables: category, film_category, inventory, payment, and rental.)

In [204]:
columns = inspector.get_columns('category')
for column in columns:
    print(column["name"], column["type"])

category_id TINYINT(3) UNSIGNED
name VARCHAR(25)
last_update TIMESTAMP


In [205]:
columns = inspector.get_columns('film_category')
for column in columns:
    print(column["name"], column["type"])

film_id SMALLINT(5) UNSIGNED
category_id TINYINT(3) UNSIGNED
last_update TIMESTAMP


In [206]:
columns = inspector.get_columns('inventory')
for column in columns:
    print(column["name"], column["type"])

inventory_id MEDIUMINT(8) UNSIGNED
film_id SMALLINT(5) UNSIGNED
store_id TINYINT(3) UNSIGNED
last_update TIMESTAMP


In [207]:
columns = inspector.get_columns('payment')
for column in columns:
    print(column["name"], column["type"])

payment_id SMALLINT(5) UNSIGNED
customer_id SMALLINT(5) UNSIGNED
staff_id TINYINT(3) UNSIGNED
rental_id INTEGER(11)
amount DECIMAL(5, 2)
payment_date DATETIME
last_update TIMESTAMP


In [208]:
columns = inspector.get_columns('rental')
for column in columns:
    print(column["name"], column["type"])

rental_id INTEGER(11)
rental_date DATETIME
inventory_id MEDIUMINT(8) UNSIGNED
customer_id SMALLINT(5) UNSIGNED
return_date DATETIME
staff_id TINYINT(3) UNSIGNED
last_update TIMESTAMP


In [209]:
result = session.query(
    Category.name,
    func.sum(Payment.amount).label('total_revenue')
) \
.join(Rental, Payment.rental_id == Rental.rental_id) \
.join(Inventory, Rental.inventory_id == Inventory.inventory_id) \
.join(Film_Category, Inventory.film_id == Film_Category.film_id) \
.join(Category, Film_Category.category_id == Category.category_id) \
.group_by(Category.category_id) \
.order_by(func.sum(Payment.amount).desc()) \
.limit(5)

In [210]:
for row in result:
    print(*row)

Sports 5314.21
Sci-Fi 4756.98
Animation 4656.30
Drama 4587.39
Comedy 4383.58
